In [1]:
##Going to do a quick EDA of the auction data + bit of data manipulation to get it ready for regression.

In [ ]:
##Look at old kaggle notebooks to look at distrubtion

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [2]:
df = pd.read_pickle("auctiondata.pkl")

In [3]:
df.head()

,itemId,title,globalId,primaryCategory.categoryId,primaryCategory.categoryName,viewItemURL,paymentMethod,autoPay,location,country,...,Operating System,Style,Connectivity,Features,Processor,Screen Size,Storage Capacity,Network,Colour,Model Number
0,274325644985,Apple iPhone 7 Plus - 128GB - Jet Black (Unloc...,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-Plus...,PayPal,False,"Corby,United Kingdom",GB,...,iOS,None,"{Lightning,Bluetooth,4G,NFC,Wi-Fi}",28,Quad Core,5.5 in,128 GB,Unlocked,Black,A1784 (GSM)
1,193490354217,Apple iPhone 7 Black 32GB (Unlocked) Please Re...,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-Blac...,PayPal,False,"Cannock,United Kingdom",GB,...,iOS,Bar,"{Lightning,Bluetooth,4G,NFC,Wi-Fi}",12,Quad Core,4.7 in,32 GB,Unlocked,Black,A1778 (GSM)
2,383676133123,Apple iPhone 7 Plus -128GB - Jet Black,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-Plus...,PayPal,False,Slovenia,SI,...,iOS,Bar,"{Lightning,Bluetooth,4G,NFC,Wi-Fi}",27,Quad Core,5.5 in,128 GB,Unlocked,Black,None
3,193490633324,Apple iPhone 7 - 32GB - Black (Unlocked) A1778...,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-32GB...,PayPal,False,"Burnley,United Kingdom",GB,...,iOS,Bar,"{Lightning,Bluetooth,NFC,Wi-Fi}",18,Quad Core,4.7 in,32 GB,Unlocked,Black,A1778 (GSM)
4,313184327229,Apple iPhone 7 Plus A1784 32GB-Unlocked-Excell...,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-Plus...,PayPal,False,"Epping,United Kingdom",GB,...,iOS,Bar,"{Lightning,Bluetooth,NFC,Wi-Fi}",19,Quad Core,None,32 GB,Unlocked,Black,None


In [4]:
corpus = []
for i in range(100):
    title = df.iloc[i]['title']
    corpus.append(f"'{title}',")

In [5]:
for title in df['title']:
    corpus.append(f"'{title}',")

In [6]:
vec = CountVectorizer(ngram_range=(1,2)).fit(corpus)
bag_of_words = vec.transform(corpus)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq_counter =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [7]:
words_freq_counter[1:10]

[('apple', 16819),
 ('apple iphone', 16525),
 ('unlocked', 11965),
 ('32gb', 11551),
 ('gsm', 9022),
 ('black', 8950),
 ('iphone 32gb', 7766),
 ('a1778', 6832),
 ('a1778 gsm', 6152)]

In [8]:
vec = TfidfVectorizer(ngram_range=(1,2)).fit(corpus)
bag_of_words = vec.transform(corpus)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [9]:
words_freq[1:10]

[('apple iphone', 1613.6096121630255),
 ('apple', 1609.8031340479592),
 ('unlocked', 1544.841805824373),
 ('32gb', 1538.9428471499652),
 ('gsm', 1512.8897992243478),
 ('iphone 32gb', 1416.0368196235856),
 ('black', 1404.1295354959127),
 ('a1778', 1359.317821654471),
 ('a1778 gsm', 1348.5659361784676)]

In [10]:
word_counts = [words_freq[0:100],words_freq_counter[0:100]]
columns = []

for i in range(len(word_counts)):
    words_freq = word_counts[i]
    for y in range(len(words_freq)):
        column = words_freq[y][0]
        columns.append(column)
        
columns_no_duplicates = list(dict.fromkeys(columns))

In [11]:
columns = []
for column_name in columns_no_duplicates:
    column_name = f'{column_name} in title'
    columns.append(column_name)

words_df = pd.DataFrame(columns=columns)
words_df['Price'] = df['sellingStatus.convertedCurrentPrice.value']
words_df['Title'] = df['title']

for index,title in enumerate(words_df['Title']):
    if(index % 1000 == 0):
        print(f'Item {index} out of {len(words_df)}')
    for word in columns_no_duplicates:
        column = f'{word} in title'
        if re.search(r'\b' + word + r'\b', title.lower()):
            words_df[column].iloc[index] = 1 
        else:
            words_df[column].iloc[index] = 0

Item 0 out of 18194


C:\Users\Alex\Miniconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Item 1000 out of 18194
Item 2000 out of 18194
Item 3000 out of 18194
Item 4000 out of 18194
Item 5000 out of 18194
Item 6000 out of 18194
Item 7000 out of 18194
Item 8000 out of 18194
Item 9000 out of 18194
Item 10000 out of 18194
Item 11000 out of 18194
Item 12000 out of 18194
Item 13000 out of 18194
Item 14000 out of 18194
Item 15000 out of 18194
Item 16000 out of 18194
Item 17000 out of 18194
Item 18000 out of 18194


In [12]:
words_df['plus in title'].value_counts()

0    13178
1     5016
Name: plus in title, dtype: int64

In [13]:
words_df.drop(columns='Title',inplace=True)

In [15]:
dummie = pd.get_dummies(words_df)
columns = dummie.columns
columns_to_keep = []
for column in columns:
    corr = dummie['Price'].corr(dummie[column])
    if(corr > 0.1 or corr < -0.1):
        print("The correlation is : {:.2f} for column {}".format(corr,column))
        columns_to_keep.append(column)
        
columns_to_keep.remove("Price")
words_in_title_to_keep = [col.split('in')[0] for col in columns_to_keep]

The correlation is : 0.23 for column unlocked in title
The correlation is : -0.17 for column 32gb in title
The correlation is : -0.19 for column a1778 in title
The correlation is : 0.16 for column 128gb in title
The correlation is : 0.32 for column plus in title
The correlation is : 0.20 for column a1784 in title
The correlation is : 0.16 for column condition in title
The correlation is : 0.11 for column plus 128gb in title
The correlation is : -0.19 for column faulty in title
The correlation is : -0.15 for column read in title
The correlation is : 0.14 for column 256gb in title
The correlation is : 0.10 for column red in title
The correlation is : -0.10 for column screen in title
The correlation is : -0.14 for column for in title
The correlation is : -0.11 for column faulty apple in title
The correlation is : -0.11 for column fault in title
The correlation is : -0.10 for column below in title
The correlation is : 0.17 for column new in title
The correlation is : -0.12 for column spare

In [19]:
for column_name in words_in_title_to_keep:
    column_name = f'{column_name} in title'
    df[column_name] = df['itemId']
    
for index,title in enumerate(df['title']):
    if(index % 1000 == 0):
        print(f'Item {index} out of {len(words_df)}')
    for word in words_in_title_to_keep:
        if re.search(r'\b' + word + r'\b', title.lower()):
            column = f'{word} in title'
            df[column].iloc[index] = 1 
        else:
            column = f'{word} in title'
            df[column].iloc[index] = 0 
            

Item 0 out of 18194
Item 1000 out of 18194
Item 2000 out of 18194
Item 3000 out of 18194
Item 4000 out of 18194
Item 5000 out of 18194
Item 6000 out of 18194
Item 7000 out of 18194
Item 8000 out of 18194
Item 9000 out of 18194
Item 10000 out of 18194
Item 11000 out of 18194
Item 12000 out of 18194
Item 13000 out of 18194
Item 14000 out of 18194
Item 15000 out of 18194
Item 16000 out of 18194
Item 17000 out of 18194
Item 18000 out of 18194


In [67]:
pd.set_option('display.max_colwidth', -1)
df.head()

,itemId,title,globalId,primaryCategory.categoryId,primaryCategory.categoryName,viewItemURL,paymentMethod,autoPay,location,country,...,read in title,256gb in title,red in title,screen in title,for in title,faulty apple in title,fault in title,below in title,new in title,spares in title
0,274325644985,Apple iPhone 7 Plus - 128GB - Jet Black (Unlocked) A1784 (GSM),EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-Plus-128GB-Jet-Black-Unlocked-A1784-GSM-/274325644985,PayPal,False,"Corby,United Kingdom",GB,...,0,0,0,0,0,0,0,0,0,0
1,193490354217,Apple iPhone 7 Black 32GB (Unlocked) Please Read Description! Slight Fault!,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-Black-32GB-Unlocked-Please-Read-Description-Slight-Fault-/193490354217,PayPal,False,"Cannock,United Kingdom",GB,...,1,0,0,0,0,0,0,0,0,0
2,383676133123,Apple iPhone 7 Plus -128GB - Jet Black,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-Plus-128GB-Jet-Black-/383676133123,PayPal,False,Slovenia,SI,...,0,0,0,0,0,0,0,0,0,0
3,193490633324,Apple iPhone 7 - 32GB - Black (Unlocked) A1778 (GSM) Apple Warranty,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-32GB-Black-Unlocked-A1778-GSM-Apple-Warranty-/193490633324,PayPal,False,"Burnley,United Kingdom",GB,...,0,0,0,0,0,0,0,0,0,0
4,313184327229,Apple iPhone 7 Plus A1784 32GB-Unlocked-Excellent condition - 97% batt. health,EBAY-GB,9355,Mobile & Smart Phones,https://www.ebay.co.uk/itm/Apple-iPhone-7-Plus-A1784-32GB-Unlocked-Excellent-condition-97-batt-health-/313184327229,PayPal,False,"Epping,United Kingdom",GB,...,0,0,0,0,0,0,0,0,0,0


In [20]:
descriptions = df['Description']
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext
cleaned_descriptions = list(map(cleanhtml,descriptions))

In [21]:
vec = CountVectorizer(ngram_range=(1,5),max_features=10000).fit(cleaned_descriptions)
bag_of_words = vec.transform(cleaned_descriptions)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq_counter =sorted(words_freq, key = lambda x: x[1], reverse=True)

vec = TfidfVectorizer(ngram_range=(1,5),max_features=10000).fit(cleaned_descriptions)
bag_of_words = vec.transform(cleaned_descriptions)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [22]:
words_freq_counter[0:10]

[('false', 192563),
 ('locked', 66668),
 ('name', 66015),
 ('lsdexception', 65199),
 ('lsdexception locked', 65199),
 ('locked false', 65199),
 ('lsdexception locked false', 65199),
 ('semihidden', 64569),
 ('unhidewhenused', 64569),
 ('priority', 63786)]

In [23]:
words_freq[0:10]

[('the', 999.6134660072645),
 ('is', 762.924405112134),
 ('and', 756.6880473217053),
 ('with', 689.4372543826692),
 ('nbsp', 665.9840847364544),
 ('condition', 631.3295932498318),
 ('to', 593.7997193771481),
 ('in', 575.306335770244),
 ('phone', 561.0228380952142),
 ('iphone', 557.704733173033)]

In [24]:
word_counts = [words_freq[0:100],words_freq_counter[0:100]]
columns = []

for i in range(len(word_counts)):
    words_freq = word_counts[i]
    for y in range(len(words_freq)):
        column = words_freq[y][0]
        columns.append(column)
        
columns_no_duplicates = list(dict.fromkeys(columns))

In [25]:
columns = []
for column_name in columns_no_duplicates:
    column_name = f'{column_name} in desc'
    columns.append(column_name)

desc_df = pd.DataFrame(columns=columns)
desc_df['Price'] = df['sellingStatus.convertedCurrentPrice.value']
desc_df['Desc'] = df['Description']

for index,desc in enumerate(desc_df['Desc']):
    if(index % 1000 == 0):
        print(f'Item {index} out of {len(desc_df)}')
    for word in columns_no_duplicates:
        column = f'{word} in desc'
        if re.search(r'\b' + word + r'\b', desc.lower()):
            desc_df[column].iloc[index] = 1 
        else:
            desc_df[column].iloc[index] = 0

Item 0 out of 18194


C:\Users\Alex\Miniconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Item 1000 out of 18194
Item 2000 out of 18194
Item 3000 out of 18194
Item 4000 out of 18194
Item 5000 out of 18194
Item 6000 out of 18194
Item 7000 out of 18194
Item 8000 out of 18194
Item 9000 out of 18194
Item 10000 out of 18194
Item 11000 out of 18194
Item 12000 out of 18194
Item 13000 out of 18194
Item 14000 out of 18194
Item 15000 out of 18194
Item 16000 out of 18194
Item 17000 out of 18194
Item 18000 out of 18194


In [26]:
desc_df['unlocked in desc'].value_counts()

1    11763
0     6431
Name: unlocked in desc, dtype: int64

In [31]:
desc_df.drop(columns='Desc',inplace=True)
dummie = pd.get_dummies(words_df)
columns = dummie.columns
columns_to_keep = []
for column in columns:
    corr = dummie['Price'].corr(dummie[column])
    if(corr > 0.1 or corr < -0.1):
        print(f"The corr : {corr} for column {column}")
        columns_to_keep.append(column)
        
columns_to_keep.remove("Price")
words_in_title_to_keep = [col.split('in')[0] for col in columns_to_keep]
words_in_title_to_keep = [x.strip() for x in words_in_title_to_keep]

The corr : 0.22819164266961542 for column unlocked in title
The corr : -0.17184855615003242 for column 32gb in title
The corr : -0.1919572007192086 for column a1778 in title
The corr : 0.16159348158292355 for column 128gb in title
The corr : 0.31605893202451274 for column plus in title
The corr : 0.19664587446287005 for column a1784 in title
The corr : 0.15692574630105083 for column condition in title
The corr : 0.11467112506428889 for column plus 128gb in title
The corr : -0.18643961382740512 for column faulty in title
The corr : -0.14543939106276144 for column read in title
The corr : 0.13705569161959133 for column 256gb in title
The corr : 0.1031727652739837 for column red in title
The corr : -0.10191166111183086 for column screen in title
The corr : -0.14426785667817554 for column for in title
The corr : -0.11434405092405431 for column faulty apple in title
The corr : -0.10564652848488985 for column fault in title
The corr : -0.10213101991657787 for column below in title
The corr :

In [33]:
words_in_title_to_keep

['unlocked',
 '32gb',
 'a1778',
 '128gb',
 'plus',
 'a1784',
 'condition',
 'plus 128gb',
 'faulty',
 'read',
 '256gb',
 'red',
 'screen',
 'for',
 'faulty apple',
 'fault',
 'below',
 'new',
 'spares']

In [34]:
for column_name in words_in_title_to_keep:
    column_name = f'{column_name} in desc'
    df[column_name] = df['itemId']
    
for index,desc in enumerate(df['Description']):
    if(index % 1000 == 0):
        print(f'Item {index} out of {len(words_df)}')
    for word in words_in_title_to_keep:
        if re.search(r'\b' + word + r'\b', desc.lower()):
            column = f'{word} in desc'
            df[column].iloc[index] = 1 
        else:
            column = f'{word} in desc'
            df[column].iloc[index] = 0 

Item 0 out of 18194
Item 1000 out of 18194
Item 2000 out of 18194
Item 3000 out of 18194
Item 4000 out of 18194
Item 5000 out of 18194
Item 6000 out of 18194
Item 7000 out of 18194
Item 8000 out of 18194
Item 9000 out of 18194
Item 10000 out of 18194
Item 11000 out of 18194
Item 12000 out of 18194
Item 13000 out of 18194
Item 14000 out of 18194
Item 15000 out of 18194
Item 16000 out of 18194
Item 17000 out of 18194
Item 18000 out of 18194


In [36]:
df['unlocked in desc'].value_counts()

1    11763
0     6431
Name: unlocked in desc, dtype: int64

In [39]:
path = os.getcwd() + "\\auctiondatacleanedtitle.pkl"
df.to_pickle(path)